In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['dipole_moments.csv', 'potential_energy.csv', 'magnetic_shielding_tensors.csv', 'structures.csv', 'scalar_coupling_contributions.csv', 'structures', 'sample_submission.csv', 'mulliken_charges.csv', 'test.csv', 'train.csv']


In [2]:
dm = pd.read_csv(r"../input/dipole_moments.csv")
train = pd.read_csv(r"../input/train.csv")
mst = pd.read_csv(r"../input/magnetic_shielding_tensors.csv")
test = pd.read_csv(r"../input/test.csv")
mc = pd.read_csv(r"../input/mulliken_charges.csv")
pe = pd.read_csv(r"../input/potential_energy.csv")
scc = pd.read_csv(r"../input/scalar_coupling_contributions.csv")
structures = pd.read_csv(r"../input/structures.csv")
ss = pd.read_csv(r"../input/sample_submission.csv")

In [3]:
#Change the name of the columns of train data so that it can be easily combined with other datas
train.columns = ['id', 'molecule_name', 'atom_index', 'atom_index_1', 'type','scalar_coupling_constant']

In [4]:
trainindex = train['atom_index'].tolist()
trainname = train['molecule_name'].tolist()

In [5]:
#loop to get first atom index
i = 0
a = []
b = 1
c = 0
for i in range (0,4658146):
    f = 1   
    if trainindex[c] == trainindex[c + 1] and trainname[c] == trainname[c+1]:
        a.append(b)
        c += 1
        i += 1
    else:
        a.append(b)
        if trainname[c]!=trainname[1+c]:
            b += 1
            o = 0
            if trainindex[c+1] > trainindex[c]:
                b = b + (trainindex[c+1]-trainindex[c])
                o += 1
            if f < trainindex[c+1+o]:
                b = (trainindex[c+1+0]-f) + b
                f += 1
        b += 1
        c += 1
        i += 1
a.append(a[4658145])

In [6]:
#loop to get the second atom index 
trainindex1 = train['atom_index_1'].tolist()
x = 0
y = 0
z = 0
w = []
v = 0
for x in range(0,4658145):
    w.append(y)
    if trainindex1[x+1]>trainindex1[x] and trainname[x] == trainname[x+1]:
        y = y+(trainindex1[x+1]-trainindex1[x])
    else:
        y = v
    if trainindex1[x+1]<=trainindex1[x]:
        v += 1
    if trainname[x+1] != trainname[x]:
        if trainindex1[x+1]<=trainindex1[x]:
            v -= 1
        v += 2
        y = v
    p = 0
    if p < trainindex1[x+1] and trainname[x+1] != trainname[x]:
        v = v + trainindex1[x+1]
        y = v
        
    x += 1
w.append(w[4658144])
w.append(w[4658145])

In [7]:
from tqdm import tqdm_notebook as tqdm
atomic_radius = {'H':0.38, 'C':0.77, 'N':0.75, 'O':0.73, 'F':0.71} # Without fudge factor

fudge_factor = 0.05
atomic_radius = {k:v + fudge_factor for k,v in atomic_radius.items()}
print(atomic_radius)

electronegativity = {'H':2.2, 'C':2.55, 'N':3.04, 'O':3.44, 'F':3.98}

#structures = pd.read_csv(structures, dtype={'atom_index':np.int8})

atoms = structures['atom'].values
atoms_en = [electronegativity[x] for x in tqdm(atoms)]
atoms_rad = [atomic_radius[x] for x in tqdm(atoms)]

structures['EN'] = atoms_en
structures['rad'] = atoms_rad

display(structures.head())

{'H': 0.43, 'C': 0.8200000000000001, 'N': 0.8, 'O': 0.78, 'F': 0.76}


,molecule_name,atom_index,atom,x,y,z,EN,rad
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,2.55,0.82
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,2.20,0.43
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,2.20,0.43
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,2.20,0.43
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,2.20,0.43


In [8]:
i_atom = structures['atom_index'].values
p = structures[['x', 'y', 'z']].values
p_compare = p
m = structures['molecule_name'].values
m_compare = m
r = structures['rad'].values
r_compare = r

source_row = np.arange(len(structures))
max_atoms = 28

bonds = np.zeros((len(structures)+1, max_atoms+1), dtype=np.int8)
bond_dists = np.zeros((len(structures)+1, max_atoms+1), dtype=np.float32)

print('Calculating bonds')

for i in tqdm(range(max_atoms-1)):
    p_compare = np.roll(p_compare, -1, axis=0)
    m_compare = np.roll(m_compare, -1, axis=0)
    r_compare = np.roll(r_compare, -1, axis=0)
    
    mask = np.where(m == m_compare, 1, 0) #Are we still comparing atoms in the same molecule?
    dists = np.linalg.norm(p - p_compare, axis=1) * mask
    r_bond = r + r_compare
    
    bond = np.where(np.logical_and(dists > 0.0001, dists < r_bond), 1, 0)
    
    source_row = source_row
    target_row = source_row + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_row = np.where(np.logical_or(target_row > len(structures), mask==0), len(structures), target_row) #If invalid target, write to dummy row
    
    source_atom = i_atom
    target_atom = i_atom + i + 1 #Note: Will be out of bounds of bonds array for some values of i
    target_atom = np.where(np.logical_or(target_atom > max_atoms, mask==0), max_atoms, target_atom) #If invalid target, write to dummy col
    
    bonds[(source_row, target_atom)] = bond
    bonds[(target_row, source_atom)] = bond
    bond_dists[(source_row, target_atom)] = dists
    bond_dists[(target_row, source_atom)] = dists

bonds = np.delete(bonds, axis=0, obj=-1) #Delete dummy row
bonds = np.delete(bonds, axis=1, obj=-1) #Delete dummy col
bond_dists = np.delete(bond_dists, axis=0, obj=-1) #Delete dummy row
bond_dists = np.delete(bond_dists, axis=1, obj=-1) #Delete dummy col

print('Counting and condensing bonds')

bonds_numeric = [[i for i,x in enumerate(row) if x] for row in tqdm(bonds)]
bond_lengths = [[dist for i,dist in enumerate(row) if i in bonds_numeric[j]] for j,row in enumerate(tqdm(bond_dists))]
n_bonds = [len(x) for x in bonds_numeric]

#bond_data = {'bond_' + str(i):col for i, col in enumerate(np.transpose(bonds))}
#bond_data.update({'bonds_numeric':bonds_numeric, 'n_bonds':n_bonds})

bond_data = {'bonds':bonds_numeric, 'n_bonds':n_bonds, 'bond_lengths':bond_lengths}
bond_df = pd.DataFrame(bond_data)
structures = structures.join(bond_df)
display(structures.head(20))

Calculating bonds



Counting and condensing bonds


,molecule_name,atom_index,atom,x,y,z,EN,rad,bonds,n_bonds,bond_lengths
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,2.55,0.82,"[1, 2, 3, 4]",4,"[1.091953, 1.0919516, 1.0919464, 1.0919476]"
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,2.20,0.43,[0],1,[1.091953]
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,2.20,0.43,[0],1,[1.0919516]
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,2.20,0.43,[0],1,[1.0919464]
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,2.20,0.43,[0],1,[1.0919476]
5,dsgdb9nsd_000002,0,N,-0.040426,1.024108,0.062564,3.04,0.80,"[1, 2, 3]",3,"[1.01719, 1.0171872, 1.0172079]"
6,dsgdb9nsd_000002,1,H,0.017257,0.012545,-0.027377,2.20,0.43,[0],1,[1.01719]
7,dsgdb9nsd_000002,2,H,0.915789,1.358745,-0.028758,2.20,0.43,[0],1,[1.0171872]
8,dsgdb9nsd_000002,3,H,-0.520278,1.343532,-0.775543,2.20,0.43,[0],1,[1.0172079]
9,dsgdb9nsd_000003,0,O,-0.034360,0.977540,0.007602,3.44,0.78,"[1, 2]",2,"[0.9621068, 0.9621068]"


In [9]:
structures.tail()

,molecule_name,atom_index,atom,x,y,z,EN,rad,bonds,n_bonds,bond_lengths
2358652,dsgdb9nsd_133885,11,H,-1.454004,-0.967309,1.459246,2.2,0.43,[2],1,[1.0882041]
2358653,dsgdb9nsd_133885,12,H,0.277779,-2.697872,0.195770,2.2,0.43,[3],1,[1.0804824]
2358654,dsgdb9nsd_133885,13,H,2.515854,-1.151784,0.527369,2.2,0.43,[4],1,[1.0805514]
2358655,dsgdb9nsd_133885,14,H,0.013699,1.199431,-1.680192,2.2,0.43,[7],1,[1.0882041]
2358656,dsgdb9nsd_133885,15,H,1.260745,-1.246754,-1.906767,2.2,0.43,[8],1,[1.0804824]


In [14]:
#find the hybridisation of each C, N and O atom
atomtype = structures['atom'].tolist()
nbonds = structures['n_bonds'].tolist()
hybrid = []
ind = 0#index
for ind in range (0,2358657):
    hyb = 0#type of hybridisation
    sp = 0
    if atomtype[ind] == 'C':
        tb = 4#theorical number of bond
        sp = 4 - nbonds[ind]
        if sp == 2:
            hyb = 'sp'
        if sp == 1:
            hyb = 'sp2'
        if sp == 0:
            hyb = 'sp3'
    if atomtype[ind] == 'H':
        hyb = 0
    if atomtype[ind] == 'N':
        tb = 3
        sp = 3 - nbonds[ind]
        if sp == 1:
            hyb = 'sp2'
        if sp == 0:
            hyb = 'sp3'
    if atomtype[ind] == 'O':
        hyb = 'sp3'
    hybrid.append(hyb)
    ind += 1
structures.insert(6,'hybrid',hybrid)
        

In [15]:
structures.columns

Index(['molecule_name', 'atom_index', 'atom', 'x', 'y', 'z', 'hybrid', 'EN',
       'rad', 'bonds', 'n_bonds', 'bond_lengths'],
      dtype='object')

In [16]:
#get structures fitted to train data
indices = a
cols =  ['atom_index', 'x', 'y', 'z','hybrid','EN', 'rad','bonds','n_bonds', 'bond_lengths_mean']
structure1 = structures.loc[indices, cols]
indices = w
cols =  ['atom_index', 'x', 'y', 'z','hybrid','EN', 'rad','bonds','n_bonds', 'bond_lengths_mean']
structure2 = structures.loc[indices, cols]
structure1.columns = ['atom_index', 'x1', 'y1', 'z1','hybrid','EN', 'rad','bonds','n_bonds', 'bond_lengths_mean']
structure2.columns = ['atom_index', 'x2', 'y2', 'z2','hybrid','EN', 'rad','bonds','n_bonds', 'bond_lengths_mean']
x1 = structure1['x1'].tolist()
y1 = structure1['y1'].tolist()
z1 = structure1['z1'].tolist()
x2 = structure2['x2'].tolist()
y2 = structure2['y2'].tolist()
z2 = structure2['z2'].tolist()
import math
distance = []
dis = 0
for dis in range (0,4658147):
    dist = math.sqrt(abs(x1[dis]-x2[dis])**2 + abs(y1[dis]-y2[dis])**2 + abs(z1[dis]-z2[dis])**2)
    distance.append(dist)
    dis += 0
train.insert(5,'distance',distance)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  import sys


In [17]:
structure3 = structure1.copy()
structure3 = structure3.drop('x1',axis = 1)
structure3 = structure3.drop('y1',axis = 1)
structure3 = structure3.drop('z1',axis = 1)
structure4 = structure2.copy()
structure4 = structure4.drop('x2',axis = 1)
structure4 = structure4.drop('y2',axis = 1)
structure4 = structure4.drop('z2',axis = 1)

In [18]:
structure4.columns = ['atom_index_1', 'hybrid1','EN1', 'rad1','bonds1', 'n_bonds1', 'bond_lengths_mean1']

In [19]:
EN = structure3['EN'].tolist()
rad = structure3['rad'].tolist() 
n_bonds = structure3['n_bonds'].tolist()
bond_length_mean = structure3['bond_lengths_mean'].tolist()
EN1 = structure4['EN1'].tolist()
rad1 = structure4['rad1'].tolist() 
n_bonds1 = structure4['n_bonds1'].tolist()
bond_length_mean1 = structure4['bond_lengths_mean1'].tolist()

In [20]:
structures.head()

,molecule_name,atom_index,atom,x,y,z,hybrid,EN,rad,bonds,n_bonds,bond_lengths
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,sp3,2.55,0.82,"[1, 2, 3, 4]",4,"[1.091953, 1.0919516, 1.0919464, 1.0919476]"
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,0,2.20,0.43,[0],1,[1.091953]
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,0,2.20,0.43,[0],1,[1.0919516]
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,0,2.20,0.43,[0],1,[1.0919464]
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,0,2.20,0.43,[0],1,[1.0919476]


In [21]:
train.insert(3,'EN',EN)

In [22]:
hyb1 = structure3['hybrid'].tolist()
hyb2 = structure4['hybrid1'].tolist() 
train.insert(4,'rad',rad)
train.insert(5,'hyb1',hyb1)
train.insert(7,'EN1',EN1)
train.insert(8,'rad1',rad1)
train.insert(9,'hyb2',hyb2)

In [25]:
train.tail(50)

,id,molecule_name,atom_index,EN,rad,hyb1,atom_index_1,EN1,rad1,hyb2,type,distance,scalar_coupling_constant
4658097,4658097,dsgdb9nsd_133884,12,2.20,0.43,0,6,2.20,0.43,0,3JHC,4.486352,1.173400
4658098,4658098,dsgdb9nsd_133884,12,2.20,0.43,0,7,2.20,0.43,0,3JHC,2.146868,1.944430
4658099,4658099,dsgdb9nsd_133884,12,2.20,0.43,0,8,2.20,0.43,0,2JHC,1.856003,0.861393
4658100,4658100,dsgdb9nsd_133884,12,2.20,0.43,0,13,2.55,0.82,sp2,3JHH,4.677866,3.062880
4658101,4658101,dsgdb9nsd_133884,12,2.20,0.43,0,17,2.55,0.82,sp3,3JHH,2.766960,4.671720
4658102,4658102,dsgdb9nsd_133884,13,2.55,0.82,sp3,2,2.20,0.43,0,3JHC,2.516210,2.971610
4658103,4658103,dsgdb9nsd_133884,13,2.55,0.82,sp3,3,2.20,0.43,0,2JHC,2.538114,2.110520
4658104,4658104,dsgdb9nsd_133884,13,2.55,0.82,sp3,4,2.20,0.43,0,1JHC,3.549288,107.016000
4658105,4658105,dsgdb9nsd_133884,13,2.55,0.82,sp3,5,2.20,0.43,0,2JHC,5.538249,4.623360
4658106,4658106,dsgdb9nsd_133884,13,2.55,0.82,sp3,6,2.20,0.43,0,3JHC,5.613902,8.854660


In [ ]:
bond1 = structure3['bonds'].tolist()
bond2 = structure4['bonds1'].tolist()